In [1]:
import glob
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
meteo_data = []
for infile in glob.glob("/content/sample_data/meteo/*.html"):
    data = pd.read_html(infile)[1]
    # store DataFrame in list
    meteo_data.append(data)
# see pd.concat documentation for more info
meteo_data = pd.concat(meteo_data)
meteo_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16635 entries, 0 to 30
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Jour                            16635 non-null  object 
 1   Unnamed: 1                      0 non-null      float64
 2   Températures et précipitations  16635 non-null  object 
 3   Avis sur la météo               16635 non-null  object 
 4   Détails                         16635 non-null  object 
dtypes: float64(1), object(4)
memory usage: 779.8+ KB


In [3]:
meteo_data.keys()

Index(['Jour', 'Unnamed: 1', 'Températures et précipitations',
       'Avis sur la météo', 'Détails'],
      dtype='object')

In [4]:
meteo_data["Détails"]

0     Météo à Brest (Finistère) le 01/02/2021
1     Météo à Brest (Finistère) le 02/02/2021
2     Météo à Brest (Finistère) le 03/02/2021
3     Météo à Brest (Finistère) le 04/02/2021
4     Météo à Brest (Finistère) le 05/02/2021
                       ...                   
26                Météo à Paris le 27/12/2020
27                Météo à Paris le 28/12/2020
28                Météo à Paris le 29/12/2020
29                Météo à Paris le 30/12/2020
30                Météo à Paris le 31/12/2020
Name: Détails, Length: 16635, dtype: object

In [5]:
detail = meteo_data["Détails"].str.split(expand=True,)
detail[2]

0     Brest
1     Brest
2     Brest
3     Brest
4     Brest
      ...  
26    Paris
27    Paris
28    Paris
29    Paris
30    Paris
Name: 2, Length: 16635, dtype: object

In [6]:
df1 = meteo_data["Températures et précipitations"].str.split(expand=True,)

In [7]:
df1

,0,1,2,3,4
0,Températures,:,7°C/11°CPrécipitations,:,4.9mm
1,Températures,:,9°C/12°CPrécipitations,:,3.3mm
2,Températures,:,8°C/10°CPrécipitations,:,29mm
3,Températures,:,9°C/11°CPrécipitations,:,4.8mm
4,Températures,:,7°C/10°CPrécipitations,:,0.4mm
...,...,...,...,...,...
26,Températures,:,7°C/8°CPrécipitations,:,27.6mm
27,Températures,:,4°C/5°CPrécipitations,:,6.4mm
28,Températures,:,3°C/5°CPrécipitations,:,10.4mm
29,Températures,:,2°C/6°CPrécipitations,:,1.4mm


In [8]:
df_precipitations = df1[4].str.split('mm', expand=True,)
P = df_precipitations[0]
P

0      4.9
1      3.3
2       29
3      4.8
4      0.4
      ... 
26    27.6
27     6.4
28    10.4
29     1.4
30     5.8
Name: 0, Length: 16635, dtype: object

In [9]:
df1[2]

0     7°C/11°CPrécipitations
1     9°C/12°CPrécipitations
2     8°C/10°CPrécipitations
3     9°C/11°CPrécipitations
4     7°C/10°CPrécipitations
               ...          
26     7°C/8°CPrécipitations
27     4°C/5°CPrécipitations
28     3°C/5°CPrécipitations
29     2°C/6°CPrécipitations
30     5°C/6°CPrécipitations
Name: 2, Length: 16635, dtype: object

In [10]:
df_temperatures = df1[2].str.split('Précipitations', expand=True,)

In [11]:
temperatures = df_temperatures[0].str.split('/', expand=True,)
temperatures

,0,1
0,7°C,11°C
1,9°C,12°C
2,8°C,10°C
3,9°C,11°C
4,7°C,10°C
...,...,...
26,7°C,8°C
27,4°C,5°C
28,3°C,5°C
29,2°C,6°C


In [12]:
T_1 = temperatures[0].str.split('°', expand=True,)
T_1

,0,1
0,7,C
1,9,C
2,8,C
3,9,C
4,7,C
...,...,...
26,7,C
27,4,C
28,3,C
29,2,C


In [13]:
T1 = T_1[0]
T1

0     7
1     9
2     8
3     9
4     7
     ..
26    7
27    4
28    3
29    2
30    5
Name: 0, Length: 16635, dtype: object

In [14]:
T_2 = temperatures[1].str.split('°', expand=True,)
T_2

,0,1
0,11,C
1,12,C
2,10,C
3,11,C
4,10,C
...,...,...
26,8,C
27,5,C
28,5,C
29,6,C


In [15]:
T2 = T_2[0]
T2

0     11
1     12
2     10
3     11
4     10
      ..
26     8
27     5
28     5
29     6
30     6
Name: 0, Length: 16635, dtype: object

In [16]:
meteo_data["temperatures_min"]=T1
meteo_data["temperatures_max"]=T2

In [17]:
meteo_data["precipitations"]=P

In [18]:
meteo_data["precipitations"].head()

0    4.9
1    3.3
2     29
3    4.8
4    0.4
Name: precipitations, dtype: object

In [19]:
meteo_data["date"] = meteo_data["Jour"]
meteo_data["opinion"] = meteo_data["Avis sur la météo"]
meteo_data["city"] = detail[2]

In [20]:
meteo_data = meteo_data.drop(columns=["Jour", "Avis sur la météo", "Unnamed: 1","Températures et précipitations","Détails"])

In [21]:
meteo_data = meteo_data[["date", "city", "temperatures_min", "temperatures_max", "precipitations", "opinion"]]
meteo_data

,date,city,temperatures_min,temperatures_max,precipitations,opinion
0,01/02/2021,Brest,7,11,4.9,défavorable
1,02/02/2021,Brest,9,12,3.3,défavorable
2,03/02/2021,Brest,8,10,29,très défavorable
3,04/02/2021,Brest,9,11,4.8,défavorable
4,05/02/2021,Brest,7,10,0.4,défavorable
...,...,...,...,...,...,...
26,27/12/2020,Paris,7,8,27.6,très défavorable
27,28/12/2020,Paris,4,5,6.4,très défavorable
28,29/12/2020,Paris,3,5,10.4,très défavorable
29,30/12/2020,Paris,2,6,1.4,très défavorable


In [22]:
meteo_data.to_csv("/content/sample_data/meteo/meteo_data.csv")